## Setting

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
pd.set_option('display.max_columns', None) 

## 2024 to figure out patter for loop

In [3]:
df_2024 = pd.read_csv("../data/raw/Total Population of Age 15 and above by Age Group, 2024.csv")

In [4]:
df_2024_raw = df_2024.copy()

In [5]:
df_2024_raw.columns.values

array(['Age Group', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'],
      dtype=object)

In [6]:
df_2024_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Age Group   11 non-null     object
 1   Unnamed: 1  14 non-null     object
 2   Unnamed: 2  12 non-null     object
 3   Unnamed: 3  12 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes


In [7]:
df_2024_raw.head(15)

,Age Group,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,Total Population of Age 15 and above by Age Group,NaN,NaN
1,NaN,2024,NaN,NaN
2,NaN,February,August,Yearly
3,15-19,22122887,22081528,-
4,20-24,22287527,22246688,-
5,25-29,22516266,22488757,-
6,30-34,22123038,22114011,-
7,35-39,21519562,21658691,-
8,40-44,20493141,20572456,-
9,45-49,19266350,19416211,-


In [8]:
df_2024_raw.columns.values

array(['Age Group', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'],
      dtype=object)

Renaming column amd dropping index

In [9]:
df_2024_proc = (
    df_2024_raw.rename(columns = {
        'Unnamed: 1' : 'February',
        'Unnamed: 2' : 'August',
        'Unnamed: 3' : 'Yearly'
    })
    .drop(index=[0, 1, 2])
    .drop(columns=['Yearly']) 
    .reset_index(drop=True)
)

In [10]:
df_2024_proc

,Age Group,February,August
0,15-19,22122887,22081528
1,20-24,22287527,22246688
2,25-29,22516266,22488757
3,30-34,22123038,22114011
4,35-39,21519562,21658691
5,40-44,20493141,20572456
6,45-49,19266350,19416211
7,50-54,17111615,17323696
8,55-59,14642145,14796109
9,60+,31915314,32673021


generating total table

In [11]:
 df_total_detail = pd.DataFrame(columns=['Year', 'February', 'August'])

In [12]:
df_total_detail

,Year,February,August


In [13]:
total_2024 = (
    df_2024_proc
    .loc[[10], ['February', 'August']]
    .assign(Year = 2024)
    .loc[:, ['Year', 'February', 'August']]
)

In [14]:
df_total_detail = pd.concat(
    [df_total_detail, total_2024],
    ignore_index=True
)

In [15]:
df_total_detail

,Year,February,August
0,2024,213997845,215371168


In [16]:
# finish the df_2024_detail

In [17]:
df_2024_detail = df_2024_proc.drop(index=10).assign(Year=2024).copy()

### 1st Check Point

In [18]:
df_2024_detail

,Age Group,February,August,Year
0,15-19,22122887,22081528,2024
1,20-24,22287527,22246688,2024
2,25-29,22516266,22488757,2024
3,30-34,22123038,22114011,2024
4,35-39,21519562,21658691,2024
5,40-44,20493141,20572456,2024
6,45-49,19266350,19416211,2024
7,50-54,17111615,17323696,2024
8,55-59,14642145,14796109,2024
9,60+,31915314,32673021,2024


In [19]:
df_total_detail

,Year,February,August
0,2024,213997845,215371168


In [20]:
df_2024_detail

,Age Group,February,August,Year
0,15-19,22122887,22081528,2024
1,20-24,22287527,22246688,2024
2,25-29,22516266,22488757,2024
3,30-34,22123038,22114011,2024
4,35-39,21519562,21658691,2024
5,40-44,20493141,20572456,2024
6,45-49,19266350,19416211,2024
7,50-54,17111615,17323696,2024
8,55-59,14642145,14796109,2024
9,60+,31915314,32673021,2024


## Loops throught year for df_all_detail

In [21]:
#config
data_dir = Path(r"../data/raw")
file_pattern = "Total Population of Age 15 and above by Age Group, {year}.csv"
start_year = 2010
end_year   = 2023
drop_top_rows = 3

detail_bucket = []

for i in range (start_year, end_year + 1):
    file_path = data_dir / file_pattern.format(year=i)

    #raw copy
    raw_df = pd.read_csv(file_path)
    globals()[f"df_{i}_raw"] = raw_df.copy()

    #proc drop 3 top row and add corresponding year
    df_proc = (
        raw_df
        .drop(index=range(drop_top_rows))
        .drop(columns = ["Unnamed: 3"], errors = 'ignore')
        .reset_index(drop=True)
    )
    df_proc["Year"] = i
    globals()[f"df_{i}_proc"] = df_proc

    #rename column
    df_proc = df_proc.rename(columns={
        "Unnamed: 1": "February",
        "Unnamed: 2": "August"
    })

    #ensure all str
    for col in ["February", "August"]:
        if col in df_proc.columns:
            df_proc[col] = (
                df_proc[col]
                .astype(str)
                .str.replace(r"[^\d\-\.]", "", regex=True)
            )
            df_proc[col] = pd.to_numeric(df_proc[col], errors="coerce").astype("Int64")

    #fill in yearly data
    df_proc["Yearly_sum"] = (df_proc["February"].fillna(0) + df_proc["August"].fillna(0)).astype("Int64")

    
    #removing total for all detail bucket
    is_total = df_proc["Age Group"].astype(str).str.contains("total", case=False, na=False)
    df_detail = df_proc.loc[~is_total].copy() # where total got copied
    globals()[f"df_{i}_detail"] = df_detail

    detail_bucket.append(df_detail)

df_all_detail = pd.concat(detail_bucket, ignore_index=True)

In [22]:
df_all_detail.head(20)

,Age Group,February,August,Year,Yearly_sum
0,15-19,23912140,22773097,2010,46685237
1,20-24,19695850,18286136,2010,37981986
2,25-29,20971807,20970795,2010,41942602
3,30-34,20192626,20747476,2010,40940102
4,35-39,18099130,18561625,2010,36660755
5,40-44,16464142,17116126,2010,33580268
6,45-49,13798683,14167142,2010,27965825
7,50-54,11491187,12052082,2010,23543269
8,55-59,8272633,8719284,2010,16991917
9,60+,18119218,18676576,2010,36795794


## Loops throught year for df_total_detail

In [23]:
total_bucket = []

for i in range(start_year, end_year +1):
    df_proc = globals()[f"df_{i}_proc"].copy()


    #rename columns
    df_proc = df_proc.rename(columns={
        "Unnamed: 1": "February",
        "Unnamed: 2": "August",
    })

    is_total = (
        df_proc["Age Group"]
        .astype(str).str.strip().str.casefold()
        .eq("total")
    )

    df_total = df_proc.loc[is_total, ["Year", "February", "August"]].copy()

    #numeric clean
    for col in ["February", "August"]:
            df_total[col] = df_total[col].astype(str).str.replace(r"[^\d\-\.]", "", regex=True)
            df_total[col] = pd.to_numeric(df_total[col], errors="coerce").astype("Int64")

    globals()[f"df_{i}_total"] = df_total
    total_bucket.append(df_total)

    # combine all years 2010–2023
    df_total_detail_new = pd.concat(total_bucket, ignore_index=True)

if "df_total_detail" in globals():
    df_total_detail = pd.concat([df_total_detail, df_total_detail_new], ignore_index=True)
else:
    df_total_detail = df_total_detail_new

df_total_detail = (
    df_total_detail
    .sort_values("Year")              # <--- sorts by Year ascending
    .drop_duplicates(subset=["Year"], keep="first")
    .reset_index(drop=True)
)

In [24]:
print(df_total_detail)

    Year   February     August
0   2010  171017416  172070339
1   2011  172007409  173851717
2   2012  175065229  176873832
3   2013  178130470  179967361
4   2014  181169972  182992204
5   2015  184599615  186100917
6   2016  187600634  189096722
7   2017  190587918  192079416
8   2018  196938738  198126553
9   2019  199785195  201185014
10  2020  202597063  203972460
11  2021  205360436  206708299
12  2022  208544086  209420383
13  2023  211588872  212587441
14  2024  213997845  215371168


In [25]:
df_total_detail.head(18)

,Year,February,August
0,2010,171017416,172070339
1,2011,172007409,173851717
2,2012,175065229,176873832
3,2013,178130470,179967361
4,2014,181169972,182992204
5,2015,184599615,186100917
6,2016,187600634,189096722
7,2017,190587918,192079416
8,2018,196938738,198126553
9,2019,199785195,201185014


## Check Point

In [26]:
df_all_detail.head(30)

,Age Group,February,August,Year,Yearly_sum
0,15-19,23912140,22773097,2010,46685237
1,20-24,19695850,18286136,2010,37981986
2,25-29,20971807,20970795,2010,41942602
3,30-34,20192626,20747476,2010,40940102
4,35-39,18099130,18561625,2010,36660755
5,40-44,16464142,17116126,2010,33580268
6,45-49,13798683,14167142,2010,27965825
7,50-54,11491187,12052082,2010,23543269
8,55-59,8272633,8719284,2010,16991917
9,60+,18119218,18676576,2010,36795794


In [27]:
df_all_detail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Age Group   140 non-null    object
 1   February    140 non-null    Int64 
 2   August      140 non-null    Int64 
 3   Year        140 non-null    int64 
 4   Yearly_sum  140 non-null    Int64 
dtypes: Int64(3), int64(1), object(1)
memory usage: 6.0+ KB


In [28]:
df_total_detail

,Year,February,August
0,2010,171017416,172070339
1,2011,172007409,173851717
2,2012,175065229,176873832
3,2013,178130470,179967361
4,2014,181169972,182992204
5,2015,184599615,186100917
6,2016,187600634,189096722
7,2017,190587918,192079416
8,2018,196938738,198126553
9,2019,199785195,201185014


In [29]:
df_total_detail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Year      15 non-null     object
 1   February  15 non-null     object
 2   August    15 non-null     object
dtypes: object(3)
memory usage: 492.0+ bytes


In [30]:
from datetime import datetime
from pathlib import Path

# make sure folder exists
cleaned_dir = Path("../data/cleaned")
cleaned_dir.mkdir(parents=True, exist_ok=True)

# timestamp
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# filenames under Data/Cleaned
detail_filename = cleaned_dir / f"bps_all_detail_2010_2024_{timestamp}.csv"
total_filename  = cleaned_dir / f"bps_total_detail_2010_2024_{timestamp}.csv"

# export
df_all_detail.to_csv(detail_filename, index=False)
df_total_detail.to_csv(total_filename, index=False)

print(f"✅ Saved detail data as {detail_filename}")
print(f"✅ Saved totals data as {total_filename}")

✅ Saved detail data as ..\data\cleaned\bps_all_detail_2010_2024_2025-08-29_16-12-12.csv
✅ Saved totals data as ..\data\cleaned\bps_total_detail_2010_2024_2025-08-29_16-12-12.csv
